# Notebook Setup

In [1]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Import Modules

In [2]:
# install required modules quietly
required_packages = ['geopandas', 'pyspark', 'azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

In [3]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import math
import json

from pyspark.sql.functions import col
import pyspark.pandas as pd
from calendar import monthrange
from datetime import datetime
from io import BytesIO

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [4]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").config(
    "spark.jars.packages", 
    "org.apache.hadoop:hadoop-azure:3.3.1,com.microsoft.azure:azure-storage:8.6.6"
    ).getOrCreate()
# Check Spark Session Information
spark

# Constant Definitions

In [5]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data"

monthly_data_filename = raw_data_dir + os.sep + 'data_monthly_v1_0.csv'
interpolated_monthly_data_filename = data_dir + os.sep + "monthly-interpolated.csv"

# Load Monthly Data

Full features from the monthly data:
```
['SITE_ID', 'year', 'month', 'time', 'TIMESTAMP', 'dataset',
'LOCATION_LAT', 'LOCATION_LONG',
'TA_F', 'VPD_F', 'P_F', 'NETRAD',
'NEE_VUT_REF', 'NEE_VUT_REF_QC', 'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF', 'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF', 'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF',
'ET', 'BESS-PAR', 'BESS-PARdiff', 'BESS-RSDN', 'CSIF-SIFdaily', 'CSIF-SIFinst', 'PET', 'Ts', 'Tmean', 'prcp', 'vpd', 'prcp-lag3', 'ESACCI-sm', 'MODIS_LC', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'EVI', 'GCI', 'NDVI', 'NDWI', 'NIRv', 'kNDVI',
'Percent_Snow', 'Fpar', 'Lai', 'LST_Day', 'LST_Night',
'SITE_IGBP', 'MODIS_IGBP','MODIS_PFT', 'koppen_sub', 'koppen', 'CO2_concentration']
```

In [6]:
included_features= ['SITE_ID', 'year', 'month', 'TIMESTAMP',
                   'ESACCI-sm',    # ESACCI Soil Moisture (%)
                   'Percent_Snow', # Percentage of snow cover (%)
                   'NDWI',      # Normalized Different Water Index (NDWI)
                   'PET',       # Potential ET (m)
                   'MODIS_PFT', # Plant Function Type
                   'MODIS_LC',  # MODIS Land Cover
                   'Ts',        # Skin temperature (K) ??
                   'LST_Day',   # Daytime land surface temperature (K)
                   'LST_Night', # Nightime land surface temperature (K)
                   'Lai',       # Leaf Area Index (LAI)
                   'Fpar',      # Fraction of photosynthetically active radiation (fPAR)
                   'CSIF-SIFdaily', # All-sky daily average SIF
                   'BESS-PAR',      # Photosynthetic Active Radiation (PAR) (W/m^2)
                   'BESS-PARdiff',  # Diffuse PAR (W/m^2)
                   'BESS-RSDN'      # Shortwave downwelling radiation (W/m^2)
                   ]
month_df = pd.read_csv(monthly_data_filename, usecols = included_features)

# only focus on target sites
month_df['date'] = pd.to_datetime(month_df['TIMESTAMP'],  format="%Y%m")
print(f"size:{month_df.shape}")
month_df.head()

size:(19015, 20)


,SITE_ID,year,month,TIMESTAMP,BESS-PAR,BESS-PARdiff,BESS-RSDN,CSIF-SIFdaily,PET,Ts,ESACCI-sm,MODIS_LC,NDWI,Percent_Snow,Fpar,Lai,LST_Day,LST_Night,MODIS_PFT,date
0,AR-SLu,2010,1,201001,154,40,336,0.20432,-0.01339,302.46967,0.15152,7,0.03542,0.00000,0.49000,1.20000,313.84000,293.58000,SH,2010-01-01
1,AR-SLu,2010,2,201002,120,46,258,0.14553,-0.00894,298.78864,0.16656,7,0.00040,0.00000,0.43000,0.90000,309.86000,292.96000,SH,2010-02-01
2,AR-SLu,2010,3,201003,107,31,231,0.10980,-0.00813,297.54816,0.16408,7,-0.02286,0.00000,0.41000,0.80000,309.18000,290.52000,SH,2010-03-01
3,AR-SLu,2010,4,201004,81,27,175,0.07673,-0.00676,291.69604,0.12402,7,-0.04202,0.00000,0.36000,0.50000,303.24000,286.34000,SH,2010-04-01
4,AR-SLu,2010,5,201005,56,19,122,0.06602,-0.00473,287.05652,0.14273,7,-0.01064,0.00000,0.37000,0.50000,296.20000,277.82000,SH,2010-05-01


In [7]:
month_df[['date','TIMESTAMP']]

,date,TIMESTAMP
0,2010-01-01,201001
1,2010-02-01,201002
2,2010-03-01,201003
3,2010-04-01,201004
4,2010-05-01,201005
...,...,...
19010,2018-08-01,201808
19011,2018-09-01,201809
19012,2018-10-01,201810
19013,2018-11-01,201811


# Check NA

In [8]:
# Check_NA
month_df.isna().sum()

SITE_ID             0
year                0
month               0
TIMESTAMP           0
BESS-PAR            0
BESS-PARdiff        0
BESS-RSDN           0
CSIF-SIFdaily     121
PET               101
Ts                101
ESACCI-sm        1610
MODIS_LC            0
NDWI              406
Percent_Snow      143
Fpar              935
Lai               935
LST_Day             0
LST_Night           0
MODIS_PFT           0
date                0
dtype: int64

In [9]:
sites = month_df['SITE_ID'].unique()
sites

array(['AR-SLu', 'AR-Vir', 'AT-Neu', 'AU-ASM', 'AU-Ade', 'AU-Cpr',
       'AU-Cum', 'AU-DaP', 'AU-DaS', 'AU-Dry', 'AU-Emr', 'AU-Fog',
       'AU-Gin', 'AU-How', 'AU-RDF', 'AU-Rig', 'AU-Stp', 'AU-TTE',
       'AU-Tum', 'AU-Wac', 'AU-Whr', 'AU-Wom', 'AU-Ync', 'BR-Sa1',
       'BR-Sa3', 'CA-Man', 'CA-NS4', 'CA-NS7', 'CA-Oas', 'CA-Obs',
       'CA-SF1', 'CA-SF3', 'CA-TP1', 'CA-TP2', 'CA-TP3', 'CA-TPD',
       'CG-Tch', 'CH-Oe1', 'CN-Cha', 'CN-Cng', 'CN-Din', 'CN-Du2',
       'CN-Ha2', 'CN-HaM', 'CN-Qia', 'CN-Sw2', 'CZ-BK2', 'DE-Lkb',
       'DE-Lnf', 'DE-Seh', 'DE-SfN', 'DE-Spw', 'DK-Eng', 'ES-Amo',
       'ES-LgS', 'FI-Jok', 'FI-Lom', 'FI-Sod', 'FR-LBr', 'FR-Pue',
       'GH-Ank', 'GL-NuF', 'GL-ZaF', 'GL-ZaH', 'IT-CA1', 'IT-CA2',
       'IT-CA3', 'IT-Col', 'IT-Cpz', 'IT-La2', 'IT-Noe', 'IT-PT1',
       'IT-Ro1', 'IT-Ro2', 'IT-SRo', 'JP-MBF', 'JP-SMF', 'MY-PSO',
       'NL-Hor', 'PA-SPn', 'PA-SPs', 'RU-Che', 'RU-Cok', 'RU-Ha1',
       'SD-Dem', 'SJ-Adv', 'SN-Dhr', 'US-Atq', 'US-Blo', 'US-C

## Filln NAN's

In [10]:
pd.options.mode.chained_assignment = None
data_df = None
for i,s in enumerate(sites):
  site_df = month_df[month_df['SITE_ID'] == s]
  site_df = site_df.reset_index()
  site_df.set_index('date', inplace=True)

  if site_df.isna().sum().sum() != 0:
    pft = site_df['MODIS_PFT'][0]
    full_cols = site_df.columns[site_df.isna().sum() == 0]
    site_df = site_df.resample('M').mean()
    site_df[['year','month', 'TIMESTAMP']].interpolate(inplace=True)
    
    interpolate_fail = False
    try:
      site_df.interpolate(method='quadratic', limit_direction='backward', inplace=True)
    except:
      interpolate_fail = True
      #print(f"{s}:")
      #display(site_df)
    
    site_df.interpolate(method='linear', limit_direction='both', inplace=True)
    site_df = site_df.reset_index()
    site_df.set_index('date', inplace=True)
    site_df['SITE_ID'] = s
    site_df['MODIS_PFT'] = pft
  
  if site_df.isna().sum().sum() != 0:
    print(f"{s}:")
    display(site_df)

  site_df.drop("index", axis=1, inplace=True)

  if type(data_df) == type(None):
    data_df = site_df
  else:
    data_df = pd.concat([data_df, site_df])

pd.options.mode.chained_assignment = 'warn'

CA-TP3:


,index,year,month,TIMESTAMP,BESS-PAR,BESS-PARdiff,BESS-RSDN,CSIF-SIFdaily,PET,Ts,ESACCI-sm,MODIS_LC,NDWI,Percent_Snow,Fpar,Lai,LST_Day,LST_Night,SITE_ID,MODIS_PFT
date,,,,,,,,,,,,,,,,,,,,
2003-05-31,1661.00000,2003.00000,5.00000,200305.00000,92.00000,45.00000,199.00000,0.16003,-0.00319,288.54004,NaN,12.00000,0.04038,0.00000,0.47000,1.20000,292.14000,279.46000,CA-TP3,CRO
2003-06-30,1660.98258,2003.18689,3.73957,200322.42835,72.27746,37.07305,156.74336,0.06710,-0.00171,279.80723,NaN,12.00000,-0.02729,15.33778,0.34694,0.62672,284.55783,272.55655,CA-TP3,CRO
2003-07-31,1660.99112,2003.35971,2.68094,200338.65232,55.40360,30.36063,120.55444,-0.01023,-0.00045,272.39225,NaN,12.00000,-0.08310,28.46250,0.24302,0.14995,278.12447,266.71675,CA-TP3,CRO
2003-08-31,1661.02663,2003.51192,1.87013,200353.06167,42.09331,25.15123,91.96437,-0.06855,0.00053,266.61256,NaN,12.00000,-0.12458,38.81821,0.16273,-0.20930,273.11564,262.19194,CA-TP3,CRO
2003-09-30,1661.08668,2003.63957,1.32142,200365.27847,32.60539,21.54095,71.53167,-0.10690,0.00122,262.57633,NaN,12.00000,-0.15106,46.20342,0.10753,-0.44507,269.62471,259.06513,CA-TP3,CRO
2003-10-31,1661.17527,2003.75119,0.99823,200376.11716,26.30731,19.28909,57.89415,-0.12784,0.00166,260.01448,NaN,12.00000,-0.16432,51.11047,0.07374,-0.57308,267.41896,257.12786,CA-TP3,CRO
2003-11-30,1661.28668,2003.83957,0.92142,200384.87847,23.60539,18.54095,51.93167,-0.13001,0.00182,259.09228,NaN,12.00000,-0.16350,53.22278,0.06353,-0.58507,266.64071,256.50513,CA-TP3,CRO
2003-12-31,1661.42835,2003.91061,1.08586,200392.14649,24.31951,19.24664,53.24671,-0.11355,0.00172,259.74826,NaN,12.00000,-0.14855,52.68116,0.07623,-0.48184,267.23806,257.15541,CA-TP3,CRO
2004-01-31,1661.59698,2003.96102,1.49811,200397.59991,28.59721,21.45534,62.16060,-0.07809,0.00134,262.03953,NaN,12.00000,-0.11925,49.37055,0.11256,-0.26109,269.25993,259.12067,CA-TP3,CRO


CG-Tch:


,index,year,month,TIMESTAMP,BESS-PAR,BESS-PARdiff,BESS-RSDN,CSIF-SIFdaily,PET,Ts,ESACCI-sm,MODIS_LC,NDWI,Percent_Snow,Fpar,Lai,LST_Day,LST_Night,SITE_ID,MODIS_PFT
date,,,,,,,,,,,,,,,,,,,,
2006-07-31,1788.00000,2006.00000,7.00000,200607.00000,60.00000,40.00000,129.00000,0.03219,-0.00420,296.46918,0.16134,9.00000,NaN,0.00000,0.19000,0.50000,307.58000,289.80000,CG-Tch,SA
2006-08-31,1789.00000,2006.00000,8.00000,200608.00000,57.00000,39.00000,122.00000,0.00000,-0.00425,296.37665,0.17328,9.00000,NaN,0.00000,0.16000,0.40000,310.36000,290.10000,CG-Tch,SA
2006-09-30,1790.00000,2006.00000,9.00000,200609.00000,58.00000,46.00000,115.00000,0.00000,-0.00432,297.62463,0.23372,9.00000,NaN,0.00000,0.15000,0.40000,315.78000,290.98000,CG-Tch,SA
2006-10-31,1791.00000,2006.00000,10.00000,200610.00000,60.00000,47.00000,114.00000,0.02433,-0.00522,299.12616,0.28374,9.00000,NaN,0.00000,0.21000,0.50000,305.02000,293.06000,CG-Tch,SA
2006-11-30,1792.00000,2006.00000,11.00000,200611.00000,87.00000,54.00000,168.00000,0.16810,-0.00427,298.81120,0.25753,9.00000,NaN,0.00000,0.43000,1.30000,306.76000,293.60000,CG-Tch,SA
2006-12-31,1793.00000,2006.00000,12.00000,200612.00000,82.00000,52.00000,163.00000,0.23642,-0.00400,298.67575,0.27133,9.00000,NaN,0.00000,0.53000,1.70000,301.14000,295.30000,CG-Tch,SA
2007-01-31,1793.64278,2006.06050,12.27972,200618.32924,74.12018,48.32427,152.25139,0.26271,-0.00396,298.45650,0.28009,9.00000,NaN,0.00000,0.56524,1.84537,296.23787,296.25233,CG-Tch,SA
2007-02-28,1793.87054,2006.17608,11.81380,200629.42158,70.64176,45.15942,149.07437,0.26177,-0.00397,298.08646,0.27355,9.00000,NaN,0.00000,0.55880,1.81616,294.29362,296.10819,CG-Tch,SA
2007-03-31,1793.73208,2006.37152,10.50241,200647.65396,70.81937,41.82727,152.78814,0.23342,-0.00404,297.48633,0.25032,9.00000,NaN,0.00000,0.50930,1.60609,294.89061,294.83669,CG-Tch,SA


GL-ZaF:


...

,index,year,month,TIMESTAMP,BESS-PAR,BESS-PARdiff,BESS-RSDN,CSIF-SIFdaily,PET,Ts,ESACCI-sm,MODIS_LC,NDWI,Percent_Snow,Fpar,Lai,LST_Day,LST_Night,SITE_ID,MODIS_PFT
date,,,,,,,,,,,,,,,,,,,,
2015-01-31,16662.00000,2015.00000,1.00000,201501.00000,70.00000,51.00000,137.00000,0.24654,-0.01791,297.82030,NaN,2.00000,0.26299,0.00000,0.74000,3.80000,298.98000,292.78000,GF-Guy,EBF
2015-02-28,16663.00000,2015.00000,2.00000,201502.00000,85.00000,57.00000,167.00000,0.34187,-0.02408,298.22382,NaN,2.00000,0.26299,0.00000,0.82000,4.90000,299.10000,293.10000,GF-Guy,EBF
2015-03-31,16664.00000,2015.00000,3.00000,201503.00000,79.00000,57.00000,153.00000,0.40645,-0.02521,298.33660,NaN,2.00000,0.26299,0.00000,0.79000,4.60000,299.84000,293.44000,GF-Guy,EBF
2015-04-30,16665.00000,2015.00000,4.00000,201504.00000,89.00000,58.00000,177.00000,0.39480,-0.02871,298.71160,NaN,2.00000,0.26299,0.00000,0.75000,3.90000,300.60000,290.26000,GF-Guy,EBF
2015-05-31,16666.00000,2015.00000,5.00000,201505.00000,73.00000,52.00000,143.00000,0.34743,-0.01804,298.25592,NaN,2.00000,0.26299,0.00000,0.74000,3.80000,299.68000,291.74000,GF-Guy,EBF
2015-06-30,16667.00000,2015.00000,6.00000,201506.00000,99.00000,52.00000,198.00000,0.34509,-0.01309,298.20258,NaN,2.00000,0.29803,0.00000,0.80000,4.70000,299.08000,293.82000,GF-Guy,EBF
2015-07-31,16668.00000,2015.00000,7.00000,201507.00000,105.00000,53.00000,213.00000,0.34722,-0.01564,298.73413,NaN,2.00000,0.31391,0.00000,0.85000,5.20000,299.72000,293.68000,GF-Guy,EBF
2015-08-31,16669.00000,2015.00000,8.00000,201508.00000,114.00000,50.00000,235.00000,0.38614,-0.01962,299.38684,NaN,2.00000,0.30912,0.00000,0.86000,5.60000,301.18000,294.02000,GF-Guy,EBF
2015-09-30,16670.00000,2015.00000,9.00000,201509.00000,121.00000,51.00000,252.00000,0.45057,-0.02886,300.35114,NaN,2.00000,0.31230,0.00000,0.84000,5.50000,303.14000,294.96000,GF-Guy,EBF


IT-Cp2:


,index,year,month,TIMESTAMP,BESS-PAR,BESS-PARdiff,BESS-RSDN,CSIF-SIFdaily,PET,Ts,ESACCI-sm,MODIS_LC,NDWI,Percent_Snow,Fpar,Lai,LST_Day,LST_Night,SITE_ID,MODIS_PFT
date,,,,,,,,,,,,,,,,,,,,
2012-05-31,17019.00000,2012.00000,5.00000,201205.00000,127.00000,42.00000,280.00000,0.28853,-0.00615,291.16740,NaN,9.00000,0.20719,0.00000,0.65000,2.50000,299.52000,283.88000,IT-Cp2,SA
2012-06-30,17018.62515,2011.97662,6.24374,201203.90548,125.16682,41.21546,276.26348,0.26940,-0.00625,293.68548,NaN,9.00000,0.16260,-0.19205,0.72478,2.96602,303.45786,287.61386,IT-Cp2,SA
2012-07-31,17018.53039,2011.96444,7.39399,201203.83780,117.78103,38.99811,259.94810,0.24849,-0.00606,295.02073,NaN,9.00000,0.13901,-0.29208,0.77178,3.19871,305.17938,289.81393,IT-Cp2,SA
2012-08-31,17018.73300,2011.96444,8.40706,201204.85087,104.81371,35.35105,230.97425,0.22641,-0.00557,295.06845,NaN,9.00000,0.13826,-0.29208,0.78799,3.17844,304.51480,290.32857,IT-Cp2,SA
2012-09-30,17019.21223,2011.97603,9.25685,201206.86029,86.95033,30.46036,190.88248,0.20394,-0.00482,293.88872,NaN,9.00000,0.15930,-0.19684,0.77436,2.91800,301.59979,289.22186,IT-Cp2,SA
2012-10-31,17020.00000,2012.00000,10.00000,201210.00000,63.00000,24.00000,137.00000,0.17958,-0.00375,291.40290,NaN,9.00000,0.20352,0.00000,0.73000,2.40000,296.24000,286.42000,IT-Cp2,SA
2012-11-30,17021.00000,2012.00000,11.00000,201211.00000,38.00000,17.00000,81.00000,0.15467,-0.00271,287.53882,NaN,9.00000,0.25318,0.20000,0.67000,1.80000,289.38000,281.74000,IT-Cp2,SA
2012-12-31,17022.00000,2012.00000,12.00000,201212.00000,30.00000,13.00000,65.00000,0.12789,-0.00287,281.92047,NaN,9.00000,0.23199,0.00000,0.66000,1.80000,284.72000,274.70000,IT-Cp2,SA
2013-01-31,17023.00000,2013.00000,1.00000,201301.00000,32.00000,14.00000,70.00000,0.11325,-0.00218,281.27120,NaN,9.00000,0.20616,0.00000,0.66000,1.60000,284.76000,275.46000,IT-Cp2,SA


DE-Hte:


,index,year,month,TIMESTAMP,BESS-PAR,BESS-PARdiff,BESS-RSDN,CSIF-SIFdaily,PET,Ts,ESACCI-sm,MODIS_LC,NDWI,Percent_Snow,Fpar,Lai,LST_Day,LST_Night,SITE_ID,MODIS_PFT
date,,,,,,,,,,,,,,,,,,,,
2009-01-31,18679.00000,2009.00000,1.00000,200901.00000,10.00000,6.00000,21.00000,NaN,-0.00095,272.89047,0.31022,17.00000,0.17859,13.80645,0.29000,0.40000,274.12000,268.30000,DE-Hte,Other
2009-02-28,18680.00000,2009.00000,2.00000,200902.00000,15.00000,10.00000,31.00000,NaN,-0.00117,273.56256,0.30168,17.00000,0.12921,8.00000,0.33000,0.50000,273.34000,268.48000,DE-Hte,Other
2009-03-31,18681.00000,2009.00000,3.00000,200903.00000,40.00000,22.00000,89.00000,NaN,-0.00257,277.50446,0.30312,17.00000,0.04990,0.00000,0.36000,0.60000,279.58000,272.10000,DE-Hte,Other
2009-04-30,18682.00000,2009.00000,4.00000,200904.00000,95.00000,34.00000,212.00000,NaN,-0.00561,283.94714,0.24812,17.00000,0.07131,0.00000,0.59000,1.60000,290.28000,278.16000,DE-Hte,Other
2009-05-31,18683.00000,2009.00000,5.00000,200905.00000,105.00000,43.00000,229.00000,NaN,-0.00659,286.09286,0.22195,17.00000,0.24402,0.00000,0.76000,3.30000,289.58000,281.16000,DE-Hte,Other
2009-06-30,18684.00000,2009.00000,6.00000,200906.00000,101.00000,50.00000,218.00000,NaN,-0.00705,287.83190,0.24192,17.00000,0.33423,0.00000,0.78000,4.00000,289.84000,283.92000,DE-Hte,Other
2009-07-31,18684.77801,2009.02561,6.72679,200909.28788,92.91714,55.34794,198.76421,NaN,-0.00723,288.84006,0.26265,17.00000,0.38471,0.03215,0.75594,4.26686,290.06439,285.79577,DE-Hte,Other
2009-08-31,18685.24482,2009.08162,7.08157,200915.24373,81.17444,58.69778,171.66553,NaN,-0.00706,288.81109,0.27555,17.00000,0.40504,0.05405,0.70513,4.20552,289.60937,286.41603,DE-Hte,Other
2009-09-30,18685.40026,2009.16477,7.07070,200923.54773,66.32588,60.03711,137.95446,NaN,-0.00658,287.79556,0.28057,17.00000,0.39601,0.06549,0.63050,3.83367,288.52214,285.82086,DE-Hte,Other


In [11]:
data_df.isna().sum()

SITE_ID             0
year                0
month               0
TIMESTAMP           0
BESS-PAR            0
BESS-PARdiff        0
BESS-RSDN           0
CSIF-SIFdaily     150
PET               125
Ts                125
ESACCI-sm        1144
MODIS_LC            0
NDWI               42
Percent_Snow        0
Fpar                0
Lai                 0
LST_Day             0
LST_Night           0
MODIS_PFT           0
dtype: int64

In [12]:
# Save interpolated monthly data checkpoint
data_df.to_csv(interpolated_monthly_data_filename);

# Feature EDA

In [13]:
month_df.describe()

,year,month,TIMESTAMP,BESS-PAR,BESS-PARdiff,BESS-RSDN,CSIF-SIFdaily,PET,Ts,ESACCI-sm,MODIS_LC,NDWI,Percent_Snow,Fpar,Lai,LST_Day,LST_Night
count,19015.00000,19015.00000,19015.00000,19015.00000,19015.00000,19015.00000,18894.00000,18914.00000,18914.00000,17405.00000,19015.00000,18609.00000,18872.00000,18080.00000,18080.00000,19015.00000,19015.00000
mean,2010.62682,6.59085,201069.27294,79.33305,33.84565,157.84491,0.15174,-0.00731,283.59837,0.25658,7.97002,0.16576,11.62064,0.48448,1.43052,289.81701,277.19649
std,5.15090,3.38791,515.04368,59.94865,24.61444,87.69293,0.14020,0.00683,10.69100,0.06502,3.77654,0.19989,26.68487,0.19973,1.12848,13.73765,9.70005
min,2001.00000,1.00000,200101.00000,0.00000,0.00000,0.00000,-0.02920,-0.05543,239.46857,0.00001,1.00000,-0.27620,0.00000,0.06000,0.10000,238.54000,237.90000
25%,2006.00000,4.00000,200612.00000,39.00000,18.00000,81.00000,0.04175,-0.00914,276.33478,0.21682,5.00000,0.02816,0.00000,0.33000,0.50000,280.38000,271.24000
50%,2011.00000,7.00000,201103.00000,77.00000,32.00000,163.00000,0.10285,-0.00554,284.15315,0.26504,9.00000,0.16970,0.00000,0.50000,1.10000,290.90000,277.92000
75%,2015.00000,10.00000,201502.00000,106.00000,45.00000,225.00000,0.23512,-0.00262,290.97623,0.30111,11.00000,0.28347,1.17136,0.65000,2.10000,298.46000,284.18000
max,2020.00000,12.00000,202012.00000,611.00000,241.00000,373.00000,0.70735,0.00004,310.28552,0.42007,17.00000,0.85476,100.00000,0.91000,6.30000,328.28000,299.34000


In [14]:
# used data with all features
month_df = pd.read_csv(monthly_data_filename, usecols=['SITE_ID', 'SITE_IGBP', 'MODIS_IGBP', 'MODIS_LC'])
group_df = month_df.groupby('SITE_ID').first()
print(f"size:{group_df.shape}")
tmp_df = group_df[group_df['SITE_IGBP'] != group_df['MODIS_IGBP']][['SITE_IGBP', 'MODIS_IGBP', 'MODIS_LC']]
print(f"size:{tmp_df.shape}")
tmp_df

size:(243, 3)
size:(162, 3)


,SITE_IGBP,MODIS_IGBP,MODIS_LC
SITE_ID,,,
AR-SLu,MF,OSH,7
AR-Vir,ENF,SAV,9
AT-Neu,GRA,MF,5
AU-ASM,SAV,CSH,6
AU-Ade,WSA,GRA,10
AU-Cpr,SAV,CSH,6
AU-Cum,EBF,SAV,9
AU-DaS,SAV,GRA,10
AU-Dry,SAV,GRA,10
